# Libs 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>")) 

import sys
import importlib

# Libraries to show results of model and build a report. Done by Russian colleagues in HC
import utilit_model_report as UMR
import utilities_am as ut_ANDREY_v2

from tqdm import tqdm, tnrange, tqdm_notebook

import gc, re
import time
from collections import Counter, defaultdict
from functools import partial
import numpy as np

# Assign libraries of stat models (algorithms)
import lightgbm as lgb
#import xgboost as xgb
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import linregress
from sklearn.isotonic import IsotonicRegression
from sklearn import neighbors
from sklearn import metrics
from sklearn.model_selection import cross_val_score


import pickle
#import cx_Oracle as orcl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import scikitplot as skplt
# from tqdm import tqdm_notebook 
# from sklearn.linear_model import LogisticRegression

# disable Anaconda warnings
import warnings
warnings.simplefilter('ignore')

# To show charts directly in Jupyter
%pylab inline

# Increase default size of charts
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

from sklearn.model_selection import (KFold as KFold_sel, cross_val_score as cross_val_score_sel, StratifiedKFold, GridSearchCV
                                    #,train_test_split
                                    )

#from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
#from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import VarianceThreshold, SelectFromModel

from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.calibration import calibration_curve

#from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
#from imblearn.pipeline import make_pipeline as make_pipeline_imb

from sklearn.model_selection import cross_val_score, train_test_split, KFold, ShuffleSplit # !!! Change to model_sel lib

from sklearn.metrics import (roc_auc_score, roc_curve, precision_recall_curve, auc, log_loss, r2_score,
                             brier_score_loss, precision_score, recall_score, f1_score, average_precision_score, 
                             confusion_matrix,classification_report, make_scorer, accuracy_score)

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from sklearn.calibration import CalibratedClassifierCV, calibration_curve

#import imblearn.under_sampling as usampl

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

import matplotlib.pyplot as plt
import seaborn as sns

import shap

import copy
import collections
from operator import attrgetter
from lightgbm import callback, Booster, Dataset
from lightgbm.compat import string_type, integer_types, range_, zip_
from lightgbm.engine import _make_n_folds, _agg_cv_result

import scipy.stats.stats as stats
import pandas.core.algorithms as algos

ModuleNotFoundError: No module named 'matplotlib'

import importlib
importlib.reload(UMR)

In [3]:
# We cast each fields of our input data set to a proper data type using the variable "typee"
typee = {'ID_CUID': 'uint32',
 'CA_F_CLX_ELIGIBLE': 'uint8',
 'CA_LIMIT_OF': 'float64',
 'CA_MAXINST_OF': 'float32',
 'CA_AVAILABLE_LIMIT': 'float64',
 'CA_GLOBAL_LIMIT': 'float64',
 'CA_OFFER_2_AVAIL': 'float64',
 'CA_TENOR_MAX': 'float64',
 'CA_TENOR_MIN': 'float64',
 'APL_F_CL_BOD01_1M': 'float16',
 'APL_F_CL_BOD02_1M': 'float16',
 'APL_F_CL_APPROVE_1M': 'float16',
 'APL_F_CL_SIGN_1M': 'float16',
 'APL_F_POS_BOD01_1M': 'float16',
 'APL_F_POS_APPROVE_1M': 'float16',
 'APL_F_POS_SIGN_1M': 'float16',
 'APL_F_CC_BOD01_1M': 'float16',
 'APL_F_CC_BOD02_1M': 'float16',
 'APL_F_CC_APPROVE_1M': 'float16',
 'APL_F_CC_SIGN_1M': 'float16',
 'APL_F_CL_BOD01_3M': 'float16',
 'APL_F_CL_BOD02_3M': 'float16',
 'APL_F_CL_APPROVE_3M': 'float16',
 'APL_F_CL_SIGN_3M': 'float16',
 'APL_F_POS_BOD01_3M': 'float16',
 'APL_F_POS_APPROVE_3M': 'float16',
 'APL_F_POS_SIGN_3M': 'float16',
 'APL_F_CC_BOD01_3M': 'float16',
 'APL_F_CC_BOD02_3M': 'float16',
 'APL_F_CC_APPROVE_3M': 'float16',
 'APL_F_CC_SIGN_3M': 'float16',
 'APL_CNT_BOD01_1M': 'float16',
 'APL_CNT_BOD01_3M': 'float16',
 'APL_CNT_BOD01_6M': 'float16',
 'APL_CNT_BOD01_9M': 'float16',
 'APL_CNT_BOD01_12M': 'float16',
 'APL_CNT_CLX_APPROVE': 'float16',
 'APL_CNT_CLW_APPROVE': 'float16',
 'APL_CNT_CLX_SIGN': 'float16',
 'APL_CNT_CLW_SIGN': 'float16',
 'APL_CNT_POS_APPROVE': 'float16',
 'APL_CNT_POS_SIGN': 'float16',
 'APL_POS_SIGN_VOL': 'float64',
 'APL_POS_SIGN_VOL_WOI': 'float64',
 'APL_CNT_CLX_ACTIVE': 'float16',
 'APL_CNT_CLW_ACTIVE': 'float16',
 'APL_CNT_POS_ACTIVE': 'float16',
 'APL_CNT_CC_ACTIVE': 'float16',
 'APL_CNT_HAS_DELIQUENCY': 'float16',
 'CM_F_APT_1M': 'float16',
 'CM_F_APT_3M': 'float16',
 'TR_F_FIRST_TRAN_POS_1M': 'uint8',
 'TR_F_FIRST_TRAN_POS_3M': 'uint8',
 'TR_F_FIRST_TRAN_1M': 'uint8',
 'TR_F_FIRST_TRAN_3M': 'uint8',
 'TR_CNT_M_FROM_TRANS_FIRST': 'float16',
 'TR_CNT_M_FROM_TRANS_LAST': 'float16',
 'CNT_WEB_APP_MONTHS_LOGIN_LAST': 'uint8',
 'CNT_WEB_APP_MONTHS_OFFER_TAB': 'uint8',
 'CM_CNT_ATTEMPT_OUT_TLM_1M': 'float16',
 'CM_CNT_ATTEMPT_OUT_TLM_3M': 'float16',
 'CM_CNT_ATTEMPT_OUT_TLM_6M': 'float16',
 'CM_CNT_ATTEMPT_OUT_TLM_9M': 'float16',
 'CM_CNT_ATTEMPT_OUT_TLM_12M': 'float16',
 'CM_CNT_ATTEMPT_OUT_TLM': 'float16',
 'CM_CNT_CONN_OUT_TLM_1M': 'float16',
 'CM_CNT_CONN_OUT_TLM_3M': 'float16',
 'CM_CNT_CONN_OUT_TLM_6M': 'float16',
 'CM_CNT_CONN_OUT_TLM_9M': 'float16',
 'CM_CNT_CONN_OUT_TLM_12M': 'float16',
 'CM_CNT_CONN_OUT_TLM': 'float16',
 'CM_CNT_CONNECTED_RPC_TLM_1M': 'float16',
 'CM_CNT_CONNECTED_RPC_TLM_3M': 'float16',
 'CM_CNT_CONNECTED_RPC_TLM_6M': 'float16',
 'CM_CNT_CONNECTED_RPC_TLM_9M': 'float16',
 'CM_CNT_CONNECTED_RPC_TLM_12M': 'float16',
 'CM_CNT_CONNECTED_RPC_TLM': 'float16',
 'CM_CNT_ATTEMPT_FAILED_TLM_1M': 'uint8',
 'CM_CNT_ATTEMPT_FAILED_TLM_3M': 'uint8',
 'CM_CNT_ATTEMPT_FAILED_TLM_6M': 'uint8',
 'CM_CNT_ATTEMPT_FAILED_TLM_9M': 'uint8',
 'CM_CNT_ATTEMPT_FAILED_TLM_12M': 'uint8',
 'CM_CNT_ATTEMPT_FAILED_TLM': 'uint8',
 'CM_CNT_ATTEMPT_TLM_1M': 'float16',
 'CM_CNT_ATTEMPT_TLM_3M': 'float16',
 'CM_CNT_ATTEMPT_TLM_6M': 'float16',
 'CM_CNT_ATTEMPT_TLM_9M': 'float16',
 'CM_CNT_ATTEMPT_TLM_12M': 'float16',
 'CM_CNT_INB_TLM_1M': 'float16',
 'CM_CNT_INB_TLM_3M': 'float16',
 'CM_CNT_INB_TLM_6M': 'float16',
 'CM_CNT_INB_TLM_9M': 'float16',
 'CM_CNT_INB_TLM_12M': 'float16',
 'CM_CNT_RESULT_REFUSED_TLM_1M': 'uint8',
 'CM_CNT_RESULT_REFUSED_TLM_3M': 'uint8',
 'CM_CNT_RESULT_REFUSED_TLM_6M': 'uint8',
 'CM_CNT_RESULT_REFUSED_TLM_9M': 'uint8',
 'CM_CNT_RESULT_REFUSED_TLM_12M': 'uint8',
 'CM_CNT_RESULT_REFUSED_TLM': 'uint8',
 'CM_CNT_SMS_CLX_1M': 'uint8',
 'CM_CNT_SMS_CLX_3M': 'uint8',
 'CM_CNT_SMS_CLX_6M': 'uint8',
 'CM_CNT_SMS_CLX_9M': 'uint8',
 'CM_CNT_SMS_CLX_12M': 'uint8',
 'DI_FAILE_1M': 'uint8',
 'DI_FAILE_3M': 'uint16',
 'DI_FAILE_6M': 'uint16',
 'DI_FAILE_9M': 'uint16',
 'DI_FAILE_12M': 'uint16',
 'DI_FAILE': 'uint16',
 'SOC_CLI_AGE': 'float16',
 'SOC_STAGE': 'float32',
 'SOC_CNT_CLI_CHILD': 'float16',
 'SOC_CLI_INCOME_TOTAL': 'float64',
 'APL_CNT_M_FROM_LAST_BOD01': 'float16',
 'SKP_CREDIT_CASE_LAST_BOD01': 'float64',
 'APL_CNT_CC_ACTIVE_CURRENT': 'float16',
 'APL_CNT_M_FROM_FIRST_ACTIVE': 'float16',
 'APL_CNT_M_FROM_LAST_ACTIVE': 'float16',
 'APL_MIN_SIGN_VOL': 'float64',
 'APL_MAX_SIGN_VOL': 'float64',
 'APL_AVG_SIGN_VOL': 'float64',
 'APL_AVG_TENOR_PLAN': 'float16',
 'APL_AVG_TENOR_FACT': 'float16',
 'APL_CNT_MFLC': 'float32',
 'APL_TOTAL_POS_DEBT_VOL_CURRENT': 'float32',
 'APL_TOTAL_CL_DEBT_VOL_CURRENT': 'float64',
 'APL_CNT_CLX_ACTIVE_CURRENT': 'float16',
 'APL_CNT_CLW_ACTIVE_CURRENT': 'float16',
 'APL_CNT_POS_ACTIVE_CURRENT': 'float16',
 'APL_MAX_POS_IR': 'float32',
 'APL_MIN_POS_IR': 'float32',
 'APL_MAX_CLX_IR': 'float32',
 'APL_MIN_CLX_IR': 'float32',
 'APL_MAX_CLW_IR': 'float16',
 'APL_MIN_CLW_IR': 'float16',
 'APL_CNT_M_FROM_APPROVE_FIRST': 'float16',
 'APL_CNT_M_FROM_APPROVE_LAST': 'float16',
 'COLL_ANNUITY_TOTAL_CURRENT': 'float32',
 'COLL_CNT_EARLY_CL': 'float16',
 'COLL_CNT_EARLY_POS': 'float16',
 'COLL_CNT_M_FROM_FIRST_DUE': 'float16',
 'COLL_CNT_M_FROM_LAST_DUE': 'float16',
 'PCB_CNT_MOB_FROM_FIRST_LOAN': 'float16',
 'PCB_CNT_MOB_FROM_LAST_LOAN': 'float16',
 'PCB_CNT_MOB_FROM_FIRST_CL': 'float16',
 'PCB_CNT_MOB_FROM_LAST_CL': 'float16',
 'PCB_CNT_MOB_FROM_FIRST_LOAN_HC': 'float16',
 'PCB_CNT_MOB_FROM_LAST_LOAN_HC': 'float16',
 'PCB_CNT_MOB_FROM_FIRST_CL_HC': 'float16',
 'PCB_CNT_MOB_FROM_LAST_CL_HC': 'float16',
 'PCB_AVG_VIETTEL_TELCO_SCORE': 'float32',
 'PCB_AVG_MOBI_TELCO_SCORE': 'float32',
 'PCB_AVG_VINA_CREDIT_SCORE': 'float32',
 'CIC_CNT_APL_ACTIVE_AT_LAST_POS': 'float16',
 'CIC_CURRENT_DEBT_VOL_AT_LAST_POS': 'float32',
 'PCB_CNT_HIST_CONTRACT_12M_BW': 'uint8',
 'PCB_CNT_HIST_NON_HOME_CONTRACT_12M_BW': 'uint8',
 'PCB_CNT_CLX_LOAN_12M_BW': 'uint8',
 'PCB_CNT_CLX_NON_HOME_LOAN_12M_BW': 'uint8',
 'PCB_CNT_ACTIVE_LOAN': 'uint8',
 'PCB_CNT_ACTIVE_LOAN_NON_HC': 'uint8',
 'PCB_CNT_HIST_CONTRACT': 'uint8',
 'PCB_CNT_HIST_NON_HOME_CONTRACT': 'uint8',
 'PCB_CNT_CLX_LOAN': 'uint8',
 'PCB_CNT_CLX_NON_HOME_LOAN': 'uint8',
 'PCB_BALANCE_VOL_REMAINING': 'float64',
 'PCB_CNT_TENOR_REMAINING': 'float16',
 'PCB_AVG_SCORE': 'float16',
 'PMT_CNT_TOTAL': 'uint8',
 'PMT_CNT_TOTAL_12M': 'uint8',
 'PMT_MAX_AMT': 'float32',
 'PMT_AVG_AMT': 'float32',
 'FLAG_CAL_CONTRACT_SIGNED_1M': 'uint8',
 'FLAG_CCX_CONTRACT_SIGNED_1M': 'uint8'}

In [4]:
chunk_size = 100000

chunk_container = pd.read_csv("MODELLING_DATA.csv", chunksize=chunk_size, dtype = typee)
for chunk in chunk_container:
    df_data = chunk
    chunk.to_csv("MODELLING_DATA_SUB.csv", index=False)
    break


In [5]:
# try to shorten the columns having long data types
for i in tqdm_notebook(df_data.columns[df_data.dtypes=='int64']):
    df_data[i] = pd.to_numeric(df_data[i], downcast='integer')

for i in tqdm_notebook(df_data.columns[df_data.dtypes=='float64']):
    df_data[i] = pd.to_numeric(df_data[i], downcast='float')

In [6]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 191 entries, ID_CUID to FLAG_CCX_CONTRACT_SIGNED_1M
dtypes: float16(99), float32(30), int8(3), object(21), uint32(1), uint8(37)
memory usage: 50.5+ MB


# Look at data

In [7]:
# Define default parameters of displaying tabular data
from IPython.display import display, Markdown
pd.options.display.max_columns = 15
pd.options.display.max_rows = 15
df_data.columns

Index(['ID_CUID', 'EFFECTIVE_DATE', 'REPORT_DATE', 'RUN_DATE', 'CA_START_DT',
       'CA_OFFER_TYPE_CODE', 'CA_F_CLX_ELIGIBLE', 'CA_LIMIT_OF',
       'CA_MAXINST_OF', 'CA_RBP_OF',
       ...
       'PCB_CNT_CLX_LOAN', 'PCB_CNT_CLX_NON_HOME_LOAN',
       'PCB_BALANCE_VOL_REMAINING', 'PCB_CNT_TENOR_REMAINING', 'PCB_AVG_SCORE',
       'SC_GR', 'FLAG_SIGN_WO_CANCEL', 'FLAG_BEFORE_REPORT',
       'FLAG_CAL_CONTRACT_SIGNED_1M', 'FLAG_CCX_CONTRACT_SIGNED_1M'],
      dtype='object', length=191)

# Preproc data 

In [8]:
#give a unique name to your model, all files created in process of modelling will have names ended on the unique_name
unique_name = '_cl'

In [9]:
#predictors from ADM to use
cols_pred=[  'CA_OFFER_TYPE_CODE', 'CA_F_CLX_ELIGIBLE', 'CA_LIMIT_OF', 'CA_MAXINST_OF', 'CA_RBP_OF', 'CA_PROD_PRI_GR', 
            'CA_TENOR_MAX', 'CA_TENOR_MIN', 'APL_F_CL_BOD01_1M', 'APL_F_CL_BOD02_1M',
           'APL_F_CL_APPROVE_1M', 'APL_F_CL_SIGN_1M', 'APL_F_POS_BOD01_1M', 'APL_F_POS_APPROVE_1M', 'APL_F_POS_SIGN_1M', 'APL_F_CC_BOD01_1M', 
           'APL_F_CC_BOD02_1M', 'APL_F_CC_APPROVE_1M', 'APL_F_CC_SIGN_1M', 'APL_F_CL_BOD01_3M', 'APL_F_CL_BOD02_3M', 'APL_F_CL_APPROVE_3M',
           'APL_F_CL_SIGN_3M', 'APL_F_POS_BOD01_3M', 'APL_F_POS_APPROVE_3M', 'APL_F_POS_SIGN_3M', 'APL_F_CC_BOD01_3M', 'APL_F_CC_BOD02_3M', 
           'APL_F_CC_APPROVE_3M', 'APL_F_CC_SIGN_3M', 'APL_CNT_BOD01_1M', 'APL_CNT_BOD01_3M', 'APL_CNT_BOD01_6M', 'APL_CNT_BOD01_9M', 
           'APL_CNT_BOD01_12M', 'APL_CNT_CLX_APPROVE', 'APL_CNT_CLW_APPROVE', 'APL_CNT_CLX_SIGN', 'APL_CNT_CLW_SIGN', 'APL_CNT_POS_APPROVE', 
           'APL_CNT_POS_SIGN', 'APL_POS_SIGN_VOL', 'APL_POS_SIGN_VOL_WOI', 'APL_CNT_CLX_ACTIVE', 'APL_CNT_CLW_ACTIVE', 'APL_CNT_POS_ACTIVE', 
           'APL_CNT_CC_ACTIVE', 'APL_CNT_HAS_DELIQUENCY', 'CM_F_APT_1M', 'CM_F_APT_3M', 'TR_F_FIRST_TRAN_POS_1M',
           'TR_F_FIRST_TRAN_POS_3M', 'TR_F_FIRST_TRAN_1M', 'TR_F_FIRST_TRAN_3M', 'TR_CNT_M_FROM_TRANS_FIRST', 'TR_CNT_M_FROM_TRANS_LAST',
           'CNT_WEB_APP_MONTHS_LOGIN_LAST', 'CNT_WEB_APP_MONTHS_OFFER_TAB', 'CM_CNT_ATTEMPT_OUT_TLM_1M', 'CM_CNT_ATTEMPT_OUT_TLM_3M', 
           'CM_CNT_ATTEMPT_OUT_TLM_6M', 'CM_CNT_ATTEMPT_OUT_TLM_9M', 'CM_CNT_ATTEMPT_OUT_TLM_12M', 'CM_CNT_ATTEMPT_OUT_TLM', 
           'CM_CNT_CONN_OUT_TLM_1M', 'CM_CNT_CONN_OUT_TLM_3M', 'CM_CNT_CONN_OUT_TLM_6M', 'CM_CNT_CONN_OUT_TLM_9M', 'CM_CNT_CONN_OUT_TLM_12M', 
           'CM_CNT_CONN_OUT_TLM', 'CM_CNT_CONNECTED_RPC_TLM_1M', 'CM_CNT_CONNECTED_RPC_TLM_3M', 'CM_CNT_CONNECTED_RPC_TLM_6M', 
           'CM_CNT_CONNECTED_RPC_TLM_9M', 'CM_CNT_CONNECTED_RPC_TLM_12M', 'CM_CNT_CONNECTED_RPC_TLM', 'CM_CNT_ATTEMPT_FAILED_TLM_1M', 
           'CM_CNT_ATTEMPT_FAILED_TLM_3M', 'CM_CNT_ATTEMPT_FAILED_TLM_6M', 'CM_CNT_ATTEMPT_FAILED_TLM_9M', 'CM_CNT_ATTEMPT_FAILED_TLM_12M', 
           'CM_CNT_ATTEMPT_FAILED_TLM', 'CM_CNT_ATTEMPT_TLM_1M', 'CM_CNT_ATTEMPT_TLM_3M', 'CM_CNT_ATTEMPT_TLM_6M', 'CM_CNT_ATTEMPT_TLM_9M',
           'CM_CNT_ATTEMPT_TLM_12M', 'CM_CNT_INB_TLM_1M', 'CM_CNT_INB_TLM_3M', 'CM_CNT_INB_TLM_6M', 'CM_CNT_INB_TLM_9M', 'CM_CNT_INB_TLM_12M',
           'CM_CNT_RESULT_REFUSED_TLM_1M', 'CM_CNT_RESULT_REFUSED_TLM_3M', 'CM_CNT_RESULT_REFUSED_TLM_6M', 'CM_CNT_RESULT_REFUSED_TLM_9M', 
           'CM_CNT_RESULT_REFUSED_TLM_12M', 'CM_CNT_RESULT_REFUSED_TLM', 'SOC_CLI_AGE', 'SOC_CLI_GENDER', 'SOC_SK_CLI_INCOME_TYPE', 'SOC_CLI_JOB', 
           'SOC_STAGE', 'SOC_CLI_INCOME_TOTAL', 'SOC_CLI_EDUCATION', 'SOC_CLI_CONTACT_REGION', 'SOC_CLI_MARITAL_STATUS'
           ,'SOC_CLI_AGE_GR', 'SOC_CLI_INCOME_GR', 'SOC_REGION_POPULATION_GR', 'SOC_CLI_CONTACT_REGION_LEVEL', 'SOC_CLI_EDUCATION_GR'
           , 'APL_CNT_M_FROM_LAST_BOD01', 'APL_STATUS_LAST_BOD01', 'APL_CNT_CC_ACTIVE_CURRENT',
           'APL_CNT_M_FROM_FIRST_ACTIVE', 'APL_CNT_M_FROM_LAST_ACTIVE', 'APL_MIN_SIGN_VOL', 'APL_MAX_SIGN_VOL', 'APL_AVG_SIGN_VOL',
           'APL_AVG_TENOR_PLAN', 'APL_AVG_TENOR_FACT', 'APL_CNT_MFLC', 'APL_TOTAL_POS_DEBT_VOL_CURRENT', 'APL_TOTAL_CL_DEBT_VOL_CURRENT', 
           'APL_CNT_CLX_ACTIVE_CURRENT', 'APL_CNT_CLW_ACTIVE_CURRENT', 'APL_CNT_POS_ACTIVE_CURRENT', 'APL_MAX_POS_IR', 'APL_MIN_POS_IR', 
           'APL_MAX_CLX_IR', 'APL_MIN_CLX_IR', 'APL_MAX_CLW_IR', 'APL_MIN_CLW_IR', 'APL_CNT_M_FROM_APPROVE_FIRST', 'APL_CNT_M_FROM_APPROVE_LAST',
           'COLL_ANNUITY_TOTAL_CURRENT', 'COLL_CNT_EARLY_CL', 'COLL_CNT_EARLY_POS', 'COLL_CNT_M_FROM_FIRST_DUE', 'COLL_CNT_M_FROM_LAST_DUE', 
           'PCB_CNT_MOB_FROM_FIRST_LOAN', 'PCB_CNT_MOB_FROM_LAST_LOAN', 'PCB_CNT_MOB_FROM_FIRST_CL', 'PCB_CNT_MOB_FROM_LAST_CL', 
           'PCB_CNT_MOB_FROM_FIRST_LOAN_HC', 'PCB_CNT_MOB_FROM_LAST_LOAN_HC', 'PCB_CNT_MOB_FROM_FIRST_CL_HC', 'PCB_CNT_MOB_FROM_LAST_CL_HC',
           'PCB_AVG_VIETTEL_TELCO_SCORE', 'PCB_AVG_MOBI_TELCO_SCORE', 'PCB_AVG_VINA_CREDIT_SCORE', 'CIC_CNT_APL_ACTIVE_AT_LAST_POS', 
           'CIC_CURRENT_DEBT_VOL_AT_LAST_POS', 'PCB_CNT_HIST_CONTRACT_12M_BW', 'PCB_CNT_HIST_NON_HOME_CONTRACT_12M_BW', 'PCB_CNT_CLX_LOAN_12M_BW',
           'PCB_CNT_CLX_NON_HOME_LOAN_12M_BW', 'PCB_CNT_ACTIVE_LOAN', 'PCB_CNT_ACTIVE_LOAN_NON_HC', 'PCB_CNT_HIST_CONTRACT', 
           'PCB_CNT_HIST_NON_HOME_CONTRACT', 'PCB_CNT_CLX_LOAN', 'PCB_CNT_CLX_NON_HOME_LOAN', 'PCB_BALANCE_VOL_REMAINING', 'PCB_CNT_TENOR_REMAINING', 
           'PCB_AVG_SCORE','PMT_CNT_TOTAL','PMT_MAX_AMT','PMT_AVG_AMT']

In [2]:
#preprocessing
cols_to_drop = []

cols_to_log = []

cols_to_fix = {}

# declare a function optimizing varchar variable to categorical ones
def prepare_cat_feat_label(df, save_dict=None, categories=None, file_name='categories' + unique_name):
    """
    
    """
    
    ret_data = df.copy() 
    # Getting varchar variables
    categorical_features = get_cat_feats(ret_data)
    categories_dic = {}
    
    if len(categorical_features)>0:
        for feature in categorical_features:
                categories_dic[feature] = pd.Categorical(ret_data[feature]).categories
                
                if categories is not None:
                     ret_data[feature] = pd.Categorical(ret_data[feature], categories=categories[feature])
                else:
                    ret_data[feature] = pd.Categorical(ret_data[feature])
                    
                ret_data[feature] = ret_data[feature].cat.codes
            
        if save_dict:
            pickle.dump(categories_dic, open('{}.dat'.format(file_name), 'wb'))
                
        print('   label encoding done...')             
            
    print ('   ...ok')
    return ret_data

# function returning a list of varchar variables
def get_cat_feats(ret_data, ids=False):
    """
    """
    
    categorical_features_objects = ret_data.columns[ret_data.dtypes=='object']
    categorical_features_SK = ret_data.filter(regex='.*SK.?_').columns

    if len(categorical_features_objects)>0:
        ret_data[categorical_features_objects]=ret_data[categorical_features_objects].apply(lambda x: x.astype(str).str.upper())

    h_added_columns=[]

    categorical_features = np.concatenate([categorical_features_objects,
                                           categorical_features_SK,
                                           h_added_columns])
    
    return list(sorted(map(lambda x: ret_data.columns.get_loc(x), categorical_features))) if ids else categorical_features

NameError: name 'unique_name' is not defined

In [11]:
# Preprocess data using the functions declared above
data_full=df_data.fillna(-1)

#data_full=prepare_data_fix_log(df_data, MAKE_LOG=False)
data_full=prepare_cat_feat_label(data_full, save_dict=True)

   label encoding done...
   ...ok


In [12]:
data_full.head()

,ID_CUID,EFFECTIVE_DATE,REPORT_DATE,RUN_DATE,CA_START_DT,CA_OFFER_TYPE_CODE,CA_F_CLX_ELIGIBLE,...,PCB_CNT_TENOR_REMAINING,PCB_AVG_SCORE,SC_GR,FLAG_SIGN_WO_CANCEL,FLAG_BEFORE_REPORT,FLAG_CAL_CONTRACT_SIGNED_1M,FLAG_CCX_CONTRACT_SIGNED_1M
0,6160752,0,0,0,11,-1.0,1,...,-1.0,-1.0,15,1,1,0,0
1,8228212,0,0,0,10,-1.0,1,...,0.0,506.0,13,1,1,0,0
2,12744249,0,0,0,1,-1.0,1,...,-1.0,0.0,6,1,1,0,0
3,10409324,0,0,0,5,-1.0,1,...,-1.0,-1.0,14,1,1,0,0
4,10218197,0,0,0,3,-1.0,1,...,0.0,494.0,14,1,1,0,0


In [13]:
# checking if we didn't lose any field while preprocessing data (compare the intial and preprocessed data sets)
len(cols_pred), len(data_full.columns), len(set(data_full.columns) & set(cols_pred))

(170, 191, 170)

In [15]:
# Defining a target attribute
IND_NAME = 'FLAG_CAL_CONTRACT_SIGNED_1M' #target
target = data_full[IND_NAME]

In [16]:
# Combine predictors and target into one data set
data = data_full[cols_pred+[IND_NAME]]

In [17]:
print(data.shape) # rows and columns
data.head() # sample of data - 5 rows by default

(100000, 171)


,CA_OFFER_TYPE_CODE,CA_F_CLX_ELIGIBLE,CA_LIMIT_OF,CA_MAXINST_OF,CA_RBP_OF,CA_PROD_PRI_GR,CA_TENOR_MAX,...,PCB_BALANCE_VOL_REMAINING,PCB_CNT_TENOR_REMAINING,PCB_AVG_SCORE,PMT_CNT_TOTAL,PMT_MAX_AMT,PMT_AVG_AMT,FLAG_CAL_CONTRACT_SIGNED_1M
0,-1.0,1,17338116.0,1015092.0,2,4,20000000.0,...,-1.0,-1.0,-1.0,4,960000.0,959500.0,0
1,-1.0,1,11000000.0,2598660.0,2,2,60000000.0,...,0.0,0.0,506.0,8,1264000.0,1261000.0,0
2,-1.0,1,200000000.0,11914759.0,0,0,200000000.0,...,-1.0,-1.0,0.0,6,3489000.0,3488833.0,0
3,-1.0,1,12000000.0,2096486.0,4,6,-1.0,...,-1.0,-1.0,-1.0,6,971000.0,971000.0,0
4,-1.0,1,15000000.0,2323125.0,2,4,20000000.0,...,0.0,0.0,494.0,4,1259000.0,1258500.0,0


In [20]:
#delete consts
i = 0
not_consts = []
for c in data.columns:
    if c != IND_NAME and list(data[c].value_counts())[0]/len(data[c]) >= 0.9999: 
        print(c)
        i += 1
    else:
        not_consts.append(c)

CA_OFFER_TYPE_CODE
CA_F_CLX_ELIGIBLE
TR_F_FIRST_TRAN_1M
CNT_WEB_APP_MONTHS_OFFER_TAB


In [22]:
data = data.loc[:, not_consts]
data.shape

(100000, 167)

In [23]:
gc.collect()

60

In [24]:
target.mean(), target.sum()

(0.00625, 625)

In [26]:
#create undersample
zeros = list(data[data[IND_NAME] == 0].index)
random.shuffle(zeros)
data_oversample = pd.concat([data[data[IND_NAME] == 1], 
                              data.loc[zeros[:600000], :]], axis=0)
print(data_oversample.shape)

(100000, 167)


In [28]:
data_oversample.tail()

,CA_LIMIT_OF,CA_MAXINST_OF,CA_RBP_OF,CA_PROD_PRI_GR,CA_TENOR_MAX,CA_TENOR_MIN,APL_F_CL_BOD01_1M,...,PCB_BALANCE_VOL_REMAINING,PCB_CNT_TENOR_REMAINING,PCB_AVG_SCORE,PMT_CNT_TOTAL,PMT_MAX_AMT,PMT_AVG_AMT,FLAG_CAL_CONTRACT_SIGNED_1M
9275,27293896.0,1925607.0,4,4,40000000.0,20000000.0,-1.0,...,0.0,0.0,470.0,7,3000.0,1481143.0,0
39171,124972360.0,5688904.0,1,4,200000000.0,100000000.0,-1.0,...,-1.0,-1.0,-1.0,17,277000.0,898588.0,0
28415,119862496.0,5456828.0,1,4,200000000.0,100000000.0,-1.0,...,0.0,0.0,473.0,14,1438000.0,1338857.0,0
37886,53000000.0,6238795.0,2,1,60000000.0,40000000.0,-1.0,...,-1.0,-1.0,-1.0,30,2209000.0,2017300.0,0
67958,15000000.0,1200192.0,5,4,20000000.0,5000000.0,-1.0,...,-1.0,-1.0,-1.0,4,852000.0,851750.0,0


In [29]:
#general splitting for train and validation datasets 
#(I don't use in CL task, instead of this I use crosstime validation - the code in below)
X_train, X_val, Y_train, Y_val = train_test_split( data_oversample.drop([IND_NAME],axis=1)
                                                    , data_oversample[IND_NAME]
                                                    , test_size=0.3, stratify=data_oversample[IND_NAME]
                                                    , random_state=7)

print ('Mean target: ', target.mean())
print ('Train', (X_train.shape, Y_train.mean(), Y_train.sum()))
print ('Valid', (X_val.shape, Y_val.mean(), Y_val.sum()))

Mean target:  0.00625
Train ((70000, 166), 0.006242857142857143, 437)
Valid ((30000, 166), 0.006266666666666667, 188)


In [30]:
X_train.head()

,CA_LIMIT_OF,CA_MAXINST_OF,CA_RBP_OF,CA_PROD_PRI_GR,CA_TENOR_MAX,CA_TENOR_MIN,APL_F_CL_BOD01_1M,...,PCB_CNT_CLX_NON_HOME_LOAN,PCB_BALANCE_VOL_REMAINING,PCB_CNT_TENOR_REMAINING,PCB_AVG_SCORE,PMT_CNT_TOTAL,PMT_MAX_AMT,PMT_AVG_AMT
10739,15000000.0,3784064.0,1,2,80000000.0,60000000.0,-1.0,...,1,4192000.0,2.0,463.0,24,2096000.0,1724958.0
9881,15000000.0,4277934.0,2,4,20000000.0,5000000.0,-1.0,...,0,-1.0,-1.0,-1.0,6,1289000.0,1288833.0
18873,15000000.0,1872998.0,3,4,20000000.0,5000000.0,-1.0,...,0,0.0,0.0,499.0,12,855000.0,703667.0
43862,96111024.0,5036000.0,0,3,-1.0,-1.0,-1.0,...,0,0.0,0.0,0.0,18,1259000.0,1164667.0
88461,15651779.0,977676.0,3,3,20000000.0,5000000.0,-1.0,...,0,4294468.0,4.0,0.0,19,960000.0,1021684.0


In [34]:
Y_train.head()

10739    0
9881     0
18873    0
43862    0
88461    0
Name: FLAG_CAL_CONTRACT_SIGNED_1M, dtype: uint8

In [35]:
X_train=X_train.reset_index(drop=True)
X_val=X_val.reset_index(drop=True)
Y_train=Y_train.reset_index(drop=True)
Y_val=Y_val.reset_index(drop=True)

# Select params

In [36]:
#change a bit original lightGBM cross validation method to see both train and test sample performance
def cv_changed(params, train_set, num_boost_round=100,
               folds=None, nfold=5, stratified=True, shuffle=True,
               metrics=None, fobj=None, feval=None, init_model=None,
               feature_name='auto', categorical_feature='auto',
               early_stopping_rounds=None, fpreproc=None,
               verbose_eval=None, show_stdv=True, seed=0,
               callbacks=None):
    """Perform the cross-validation with given paramaters.

    Parameters
    ----------
    params : dict
        Parameters for Booster.
    train_set : Dataset
        Data to be trained on.
    num_boost_round : int, optional (default=100)
        Number of boosting iterations.
    folds : generator or iterator of (train_idx, test_idx) tuples, scikit-learn splitter object or None, 
        optional (default=None)
        If generator or iterator, it should yield the train and test indices for each fold.
        If object, it should be one of the scikit-learn splitter classes
        (http://scikit-learn.org/stable/modules/classes.html#splitter-classes)
        and have ``split`` method.
        This argument has highest priority over other data split arguments.
    nfold : int, optional (default=5)
        Number of folds in CV.
    stratified : bool, optional (default=True)
        Whether to perform stratified sampling.
    shuffle : bool, optional (default=True)
        Whether to shuffle before splitting data.
    metrics : string, list of strings or None, optional (default=None)
        Evaluation metrics to be monitored while CV.
        If not None, the metric in ``params`` will be overridden.
    fobj : callable or None, optional (default=None)
        Custom objective function.
    feval : callable or None, optional (default=None)
        Customized evaluation function.
        Should accept two parameters: preds, train_data,
        and return (eval_name, eval_result, is_higher_better) or list of such tuples.
        For multi-class task, the preds is group by class_id first, then group by row_id.
        If you want to get i-th row preds in j-th class, the access way is preds[j * num_data + i].
        To ignore the default metric corresponding to the used objective,
        set ``metrics`` to the string ``"None"``.
    init_model : string, Booster or None, optional (default=None)
        Filename of LightGBM model or Booster instance used for continue training.
    feature_name : list of strings or 'auto', optional (default="auto")
        Feature names.
        If 'auto' and data is pandas DataFrame, data columns names are used.
    categorical_feature : list of strings or int, or 'auto', optional (default="auto")
        Categorical features.
        If list of int, interpreted as indices.
        If list of strings, interpreted as feature names (need to specify ``feature_name`` as well).
        If 'auto' and data is pandas DataFrame, pandas categorical columns are used.
        All values in categorical features should be less than int32 max value (2147483647).
        Large values could be memory consuming. Consider using consecutive integers starting from zero.
        All negative values in categorical features will be treated as missing values.
    early_stopping_rounds : int or None, optional (default=None)
        Activates early stopping.
        CV score needs to improve at least every ``early_stopping_rounds`` round(s)
        to continue.
        Requires at least one metric. If there's more than one, will check all of them.
        To check only the first metric you can pass in ``callbacks``
        ``early_stopping`` callback with ``first_metric_only=True``.
        Last entry in evaluation history is the one from the best iteration.
    fpreproc : callable or None, optional (default=None)
        Preprocessing function that takes (dtrain, dtest, params)
        and returns transformed versions of those.
    verbose_eval : bool, int, or None, optional (default=None)
        Whether to display the progress.
        If None, progress will be displayed when np.ndarray is returned.
        If True, progress will be displayed at every boosting stage.
        If int, progress will be displayed at every given ``verbose_eval`` boosting stage.
    show_stdv : bool, optional (default=True)
        Whether to display the standard deviation in progress.
        Results are not affected by this parameter, and always contain std.
    seed : int, optional (default=0)
        Seed used to generate the folds (passed to numpy.random.seed).
    callbacks : list of callables or None, optional (default=None)
        List of callback functions that are applied at each iteration.
        See Callbacks in Python API for more information.

    Returns
    -------
    eval_hist : dict
        Evaluation history.
        The dictionary has the following format:
        {'metric1-mean': [values], 'metric1-stdv': [values],
        'metric2-mean': [values], 'metric2-stdv': [values],
        ...}.
    """
    if not isinstance(train_set, Dataset):
        raise TypeError("Traninig only accepts Dataset object")

    params = copy.deepcopy(params)
    if fobj is not None:
        params['objective'] = 'none'
    for alias in ["num_iterations", "num_iteration", "n_iter", "num_tree", "num_trees",
                  "num_round", "num_rounds", "num_boost_round", "n_estimators"]:
        if alias in params:
            warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
            num_boost_round = params.pop(alias)
            break
    for alias in ["early_stopping_round", "early_stopping_rounds", "early_stopping"]:
        if alias in params:
            warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
            early_stopping_rounds = params.pop(alias)
            break

    if num_boost_round <= 0:
        raise ValueError("num_boost_round should be greater than zero.")
    if isinstance(init_model, string_type):
        predictor = _InnerPredictor(model_file=init_model, pred_parameter=params)
    elif isinstance(init_model, Booster):
        predictor = init_model._to_predictor(dict(init_model.params, **params))
    else:
        predictor = None
    train_set._update_params(params) \
             ._set_predictor(predictor) \
             .set_feature_name(feature_name) \
             .set_categorical_feature(categorical_feature)

    if metrics is not None:
        params['metric'] = metrics

    results = collections.defaultdict(list)
    cvfolds = _make_n_folds(train_set, folds=folds, nfold=nfold,
                            params=params, seed=seed, fpreproc=fpreproc,
                            stratified=stratified, shuffle=shuffle)

    # setup callbacks
    if callbacks is None:
        callbacks = set()
    else:
        for i, cb in enumerate(callbacks):
            cb.__dict__.setdefault('order', i - len(callbacks))
        callbacks = set(callbacks)
    if early_stopping_rounds is not None:
        callbacks.add(callback.early_stopping(early_stopping_rounds, verbose=False))
    if verbose_eval is True:
        callbacks.add(callback.print_evaluation(show_stdv=show_stdv))
    elif isinstance(verbose_eval, integer_types):
        callbacks.add(callback.print_evaluation(verbose_eval, show_stdv=show_stdv))

    callbacks_before_iter = {cb for cb in callbacks if getattr(cb, 'before_iteration', False)}
    callbacks_after_iter = callbacks - callbacks_before_iter
    callbacks_before_iter = sorted(callbacks_before_iter, key=attrgetter('order'))
    callbacks_after_iter = sorted(callbacks_after_iter, key=attrgetter('order'))

    for i in range_(num_boost_round):
        for cb in callbacks_before_iter:
            cb(callback.CallbackEnv(model=cvfolds,
                                    params=params,
                                    iteration=i,
                                    begin_iteration=0,
                                    end_iteration=num_boost_round,
                                    evaluation_result_list=None))
        cvfolds.update(fobj=fobj)
        res = [_agg_cv_result(cvfolds.eval_valid(feval)), _agg_cv_result(cvfolds.eval_train(feval))]

        for _, key, mean, _, std in res[0]:
            results[key + '-val-mean'].append(mean)
            results[key + '-val-stdv'].append(std)
        for _, key, mean, _, std in res[1]:
            results[key + '-train-mean'].append(mean)
            results[key + '-train-stdv'].append(std)
            
        try:
            for cb in callbacks_after_iter:
                cb(callback.CallbackEnv(model=cvfolds,
                                        params=params,
                                        iteration=i,
                                        begin_iteration=0,
                                        end_iteration=num_boost_round,
                                        evaluation_result_list=res[0]))
        except callback.EarlyStopException as earlyStopException:
            cvfolds.best_iteration = earlyStopException.best_iteration + 1
            for k in results:
                results[k] = results[k][:cvfolds.best_iteration]
            break
    
    return dict(results)

In [37]:
lgb.cv = cv_changed

In [2]:
# functions for creating model and selection of hyperparameters
def objective_lgb(space, X, Y, cat_feats, NFALG_PRINT_HIST, NUM_FOLDS):
    #global iteration, best_auc_so_far, best_ntrees
    global iteration, best_auc_val_so_far, best_auc_val_std_so_far, best_ntrees, best_auc_train_so_far, best_auc_train_std_so_far
    iteration += 1   
    ### if NFALG_PRINT_HIST==False: print('Num step: {}'.format(iteration), end="\r")
    params = {'num_leaves': int(space['num_leaves']),
              'min_data_in_leaf': int(space['min_data_in_leaf']), 
              'max_depth': int(space['max_depth']),    
              'bagging_freq': int(space['bagging_freq']),    
              'max_bin': int(space['max_bin']),    
              
              'learning_rate': space['learning_rate'], 
              'feature_fraction': space['feature_fraction'],             
              'bagging_fraction': space['bagging_fraction'],
              'n_jobs': space['n_jobs'],
              'objective': space['objective'],
              'random_state': space['random_state']
             }

    ds = lgb.Dataset(X, 
                     Y, 
                     categorical_feature=cat_feats)
    
    cv_res = lgb.cv(params,
                    ds, 
                    metrics='auc',
                    num_boost_round=1000,
                    early_stopping_rounds=50,
                    nfold=len(time_folds) if time_folds else NUM_FOLDS,
                    folds=time_folds if time_folds else None,
                    categorical_feature=cat_feats)
    
    auc_val = cv_res['auc-val-mean'][-1]  #AUC  for the last boosting of Validation sample
    auc_val_std = cv_res['auc-val-stdv'][-1] #AUC STD for the last boosting of Validation sample
    auc_train = cv_res['auc-train-mean'][-1]  #AUC for the last boosting of Training Sample
    auc_train_std = cv_res['auc-train-stdv'][-1] #AUC STD for the last boosting of Training Sample
    
    # choosing best parameters based on a comparison between a new calculated and the last best
    if (((auc_val-2*auc_val_std)-0.5*abs((auc_train)-(auc_val))) > 
        ((best_auc_val_so_far-best_auc_val_std_so_far)-
         0.5*abs((best_auc_train_so_far)-(best_auc_val_so_far)))):
        best_auc_val_so_far = auc_val
        best_auc_val_std_so_far = auc_val_std
        best_ntrees = len(cv_res['auc-val-mean'])
        best_auc_train_so_far = auc_train
        best_auc_train_std_so_far = auc_train_std

    if NFALG_PRINT_HIST:
        print('iteration {}'.format(iteration))
        print("---AUC SCORE (val):\t{} (std: {}) ({} (std: {}) best so far)".format(round(auc_val,6), round(auc_val_std,6), 
                                                                                    round(best_auc_val_so_far,6), round(best_auc_val_std_so_far,6)))
        print("---AUC SCORE (train):\t{} (std: {}) ({} (std: {}) best so far)".format(round(auc_train,6), round(auc_train_std,6), 
                                                                                      round(best_auc_train_so_far,6), round(best_auc_train_std_so_far,6)))
        print("---ntrees:\t{} ({} best so far)\n".format(len(cv_res['auc-val-mean']), best_ntrees))
    
    return{'loss':1-((auc_val-auc_val_std)-0.5*abs((auc_train+auc_train_std)-(auc_val-auc_val_std))), 'status': STATUS_OK}

def run_hyperopt_lgb(x_train, y_train, x_test, y_test, cat_feats, evals=10, NFALG_PRINT_HIST=True, NUM_FOLDS = 3):
    iteration = 0
    
    params_space = {'num_leaves':[4, 12, 50, 100, 150, 200],
                    'min_data_in_leaf':[20, 50, 100, 250],
                    'max_depth':[2, 4, 6], # [2, 4, 6, 8, 10, 12, 14, 25]
                    'bagging_freq':[1, 2, 3, 5, 10],
                    'max_bin':[100, 255, 500],
                    
                    #'learning_rate':[0.01, 0.02, 0.03, 0.05, 0.1, 0.3, 0.5, 0.8, 1],
                    'learning_rate':[.01, .05, .1, .3, .5, .8],
                    'feature_fraction':[.25, .3, .5, .7],
                    'bagging_fraction':[.3, .5, .7],
                    'n_jobs': [8],
                    'objective': ['binary'], 
                    'random_state': [123]}
    # Variable for a final choice of the parameters (inputs) of the algorithm LightGBM
    space = {'num_leaves' : hp.choice('num_leaves', params_space['num_leaves']),
             'min_data_in_leaf' : hp.choice('min_data_in_leaf', params_space['min_data_in_leaf']),
             'max_depth' : hp.choice('max_depth', params_space['max_depth']),
             'bagging_freq' : hp.choice('bagging_freq', params_space['bagging_freq']),
             'max_bin' : hp.choice('max_bin', params_space['max_bin']),             
             
             'learning_rate' : hp.choice('learning_rate', params_space['learning_rate']),
             'feature_fraction' : hp.choice('feature_fraction', params_space['feature_fraction']),
             'bagging_fraction' : hp.choice('bagging_fraction', params_space['bagging_fraction']),
             'n_jobs' : hp.choice('n_jobs', params_space['n_jobs']),
             'objective' : hp.choice('objective', params_space['objective']),
             'random_state' : hp.choice('random_state', params_space['random_state'])}

    trials = Trials() # hyperOpt variable for choosing algorithm for best parameters finding
    # Running optimization for finding best parameters 
    best_params = fmin(fn=partial(objective_lgb, X=x_train, Y=y_train, cat_feats=cat_feats, 
                                  NFALG_PRINT_HIST=NFALG_PRINT_HIST, NUM_FOLDS=NUM_FOLDS),#objective
               space=space,
                algo=tpe.suggest, # Algorithm of finding optimal parameters. Please check how it works
                max_evals=evals, # number of iterations
                trials=trials) # object for Hyper Optimization

    print("--Best params metrics AUC SCORE:{} (std: {})  ntrees: {}".format(round(best_auc_val_so_far,6), round(best_auc_val_std_so_far,6), best_ntrees))
    
    for k in params_space.keys():  # востанавливает лучшие значения по их индексам
        best_params[k] = params_space[k][best_params[k]] # best params in a different format
            
    train = lgb.Dataset(x_train,
                        y_train,
                        categorical_feature=cat_feats) #  categorical_feature 
    
    test = lgb.Dataset(x_test,
                       y_test, 
                       categorical_feature=cat_feats)
    
    booster = lgb.train(best_params,   # calling the algorithm itself
                        train_set=train,
                        valid_sets=[train, test],
                        num_boost_round=best_ntrees,
                        categorical_feature=cat_feats,
                        verbose_eval=True)

    return booster, best_params

In [39]:
# the first model
cat_feat = get_cat_feats(X_train, ids=True)
iteration=0
best_auc_val_so_far=0
best_auc_val_std_so_far=0
best_auc_train_so_far=0
best_auc_train_std_so_far=0
time_folds=False
booster, best_params=run_hyperopt_lgb(X_train, Y_train, X_val, Y_val, cat_feat, evals=5)

iteration 1                                          
---AUC SCORE (val):	0.689065 (std: 0.056219) (0.689065 (std: 0.056219) best so far)
---AUC SCORE (train):	0.731784 (std: 0.038876) (0.731784 (std: 0.038876) best so far)
---ntrees:	3 (3 best so far)                         

iteration 2                                                                    
---AUC SCORE (val):	0.709825 (std: 0.007532) (0.709825 (std: 0.007532) best so far)
---AUC SCORE (train):	0.742676 (std: 0.011837) (0.742676 (std: 0.011837) best so far)
---ntrees:	2 (2 best so far)                                                   

iteration 3                                                                     
---AUC SCORE (val):	0.664514 (std: 0.038352) (0.709825 (std: 0.007532) best so far)
---AUC SCORE (train):	0.693287 (std: 0.029329) (0.742676 (std: 0.011837) best so far)
---ntrees:	1 (2 best so far)                                                    

iteration 4                                              

In [40]:
#performance at the validation set in comparison with the test one
y_pred_train = booster.predict(X_train)
y_pred_val = booster.predict(X_val)

print('roc auc score on train: ', roc_auc_score(Y_train, y_pred_train))
print('roc auc score on val: ', roc_auc_score(Y_val, y_pred_val))

roc auc score on train:  0.7104499153278933
roc auc score on val:  0.6737101260095179


In [ ]:
#model performance visualization (using a Russian library)
ut_ANDREY_v2.print_lift_roc_pr_plot(Y_val, y_pred_val,NAME_MODEL='val')
ut_ANDREY_v2.print_lift_roc_pr_plot(Y_train, y_pred_train,NAME_MODEL='train')

In [ ]:
#clean some memory
gc.collect()

In [ ]:
# Preparations to show most important features
shap.initjs()
model = booster
X = pd.DataFrame(X_train) 
count_show_features = 20 

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values[0], X, max_display=count_show_features)

# Select features 

In [ ]:
def plot_heatmap(df, used_feats):
    cr1 = df[used_feats].corr()
    plt.figure(figsize=(40,34))
    cmap = sns.diverging_palette(20, 220, sep=20, as_cmap=True)
    sns.heatmap(cr1, cmap=cmap, vmin=-1, vmax=1)

##  delete features with strong correlation

In [ ]:
# Function for calculating AUC for each feature
def get_aucs(mdl = LogisticRegression(n_jobs=10,C=1) , x_df = pd.DataFrame(), Y_df = pd.Series()):
    base_model = mdl
    X_Train_AUC=[]
    for idx, name in  tqdm_notebook(enumerate(x_df.columns), ncols=x_df.shape[1]):
        skf = StratifiedKFold(n_splits=5, random_state=999, shuffle=True)
        cv_score = cross_val_score(base_model
                                      ,StandardScaler().fit_transform(np.array(x_df[name]).reshape(-1, 1))
                                      ,Y_df
                                      ,n_jobs=10,cv=5,verbose=0,scoring='roc_auc')
        auc_avg, auc_std_95, auc_arr = np.mean(cv_score), cv_score.std(), cv_score #np.std(cv_score)*2 
        X_Train_AUC.append([name, auc_avg, auc_std_95, auc_arr])
    i_ind=[item[0] for item in X_Train_AUC]
    X_Train_AUC=pd.DataFrame(X_Train_AUC, columns=['NAME','AUC_AVG', 'AUC_STD_95', 'AUC_ARR'], index=i_ind)
    return X_Train_AUC

In [ ]:
# Procedure deleting high-correlated features
def correlation_to_del(dataset, threshold, df_significant, name_col_sign = ''):
    start = time.time()
    dataset_in=dataset.copy()
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset_in.corr().abs()
    for i in tqdm_notebook(range(len(corr_matrix.columns))):
        for j in range(i):
            if corr_matrix.iloc[i, j] >= threshold:
                colname_row = corr_matrix.index[j] # getting the name of index(row name)
                colname_col = corr_matrix.columns[i] # getting the name of column
                if df_significant.loc[colname_row][name_col_sign]> df_significant.loc[colname_col][name_col_sign]:
                    colname=colname_col
                else:
                    colname=colname_row
                col_corr.add(colname)
                if colname in dataset_in.columns:
                    del dataset_in[colname] # deleting the column from the dataset
    end = time.time()
    print (end-start)
    return dataset_in, col_corr, corr_matrix

In [ ]:
X_Train_AUC = get_aucs(x_df=X_train, Y_df=Y_train)

In [ ]:
gc.collect()

### select threshold 

In [ ]:
def choose_threshold(params, X, y, list_of_thresholds=None):
    """
    the function creates a visulization of model performance on cross validation for different levels of correlation 
    
    params - model's hyperparameters
    X - sample
    y - target
    list_of_thresh_holds - possible thresholds
    
    returns thresholds and related performance
    """
    list_of_thresholds = list_of_thresholds or [ 0.65, 0.70, 0.75, 0.80,0.90,0.95]
    x_aucs = []
    cnt_of_feat = []
    for threshold in list_of_thresholds:

        X_Train_NCR, col_corr, corr_matrix=correlation_to_del(X_train, threshold, X_Train_AUC, 'AUC_AVG')
        
        X_attr = X[X_Train_NCR.columns]
        cat_feat_attr = get_cat_feats(X_attr, ids=True)
    
        
        ds = lgb.Dataset(X_attr, #features
                         y, #target
                         categorical_feature=cat_feat_attr)
        
        #params['random_state'] = 123
        cv_res = lgb.cv(params,
                    ds,#что  
                    metrics='auc',
                    num_boost_round=5000,# количество деревьек, если что сработает early stoping 
                    early_stopping_rounds=50,
                    nfold=len(time_folds) if time_folds else NUM_FOLDS,
                    folds=time_folds if time_folds else None,
                    seed=123,
                    categorical_feature=cat_feat_attr)
        
        auc = cv_res['auc-val-mean'][-1]
        
        cnt_of_feat.append(len(X_attr.columns))
        x_aucs.append(auc)
        
    for i in range(len(x_aucs)):
        print('threshold: ', list_of_thresholds[i])
        print('count of features: ', cnt_of_feat[i])
        print('roc auc: ', round(x_aucs[i],3))
        print()
#     print(x_aucs) 
#     print(cnt_of_feat)
    plt.figure(figsize=(10,8))
    plt.title('Roc_auc and thresh_holds of corr', fontsize=20)
    plt.xlabel('thresh_holds of corr')
    plt.ylabel('Roc auc score')
    
    plt.plot(list_of_thresholds, x_aucs, marker='.')

    return list_of_thresholds[np.argmax(x_aucs)], max(x_aucs)

In [ ]:
NUM_FOLDS=3 # if it not used, then delete
th, m = choose_threshold(best_params, X_train, Y_train)

In [ ]:
th

In [ ]:
X_Train_NCR, col_corr, corr_matrix=correlation_to_del(X_train, th, X_Train_AUC, 'AUC_AVG')

In [ ]:
#selected features
attr_features = X_Train_NCR.columns
attr_features

In [ ]:
# the second model
cat_feat_attr = get_cat_feats(X_train[attr_features], ids=True)
iteration=0
best_auc_val_so_far=0
best_auc_val_std_so_far=0
best_auc_train_so_far=0
best_auc_train_std_so_far=0
booster_attr,best_params_attr = run_hyperopt_lgb(X_train[attr_features],  Y_train, X_val[attr_features], Y_val, cat_feat_attr, evals=10)

In [ ]:
y_pred_train_attr = booster_attr.predict(X_train[attr_features])
y_pred_val_attr = booster_attr.predict(X_val[attr_features])

print('roc auc score on train: ', roc_auc_score(Y_train, y_pred_train_attr))
print('roc auc score on val: ', roc_auc_score(Y_val, y_pred_val_attr))

In [ ]:
ut_ANDREY_v2.print_lift_roc_pr_plot(Y_val, y_pred_val_attr,NAME_MODEL='val')
ut_ANDREY_v2.print_lift_roc_pr_plot(Y_train, y_pred_train_attr,NAME_MODEL='train')

In [ ]:
gc.collect()

Two models are now available: booster on all features (X_train_1) and booster_attr on attr_features (X_train_1 [attr_features])

## Logistic Regression 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
def feats_from_log_reg(X, y, X_val=None, y_val=None, C=0.1):
    """
    The function builds a logistic regression with L1 regularizations,
    builds a graph and returns features sorted in descending order (abs)
    
    X - Training sample
    y - Answers to it
    X_val - Test sample (what if good quality) 
    y_val - Answers to it
    C= 0.1
    
    """
    
    
    scaler = StandardScaler()
    clfLR = LogisticRegression(penalty='l1', n_jobs=8, C=0.1,solver ='saga')

    pipeline = Pipeline([
        ('prepr', scaler),
        ('clf', clfLR)
    ])

    pipeline.fit(X,y )
    coefs_l1 = pipeline.named_steps['clf'].coef_[0]

    coefs_import_df = pd.DataFrame(list(zip(X.columns, coefs_l1)), columns=['feature', 'coef']).sort_values('coef', ascending=False).reset_index(drop=True)
    if X_val is not None:
        lr_pred = pipeline.predict_proba(X_val)[:,1]
        print('roc auc score on val: ',roc_auc_score(y_val, lr_pred))
    
    plt.figure(figsize=(20, 10))
    sns.barplot(x="coef", y="feature", data=coefs_import_df)
    plt.title('Logistic Regression Features')
    plt.tight_layout()
    plt.show()
    
    coefs_import_df['coef'] = coefs_import_df['coef'].apply(abs)
    
    return coefs_import_df.sort_values('coef', ascending=False)['feature'].values

In [ ]:
feats_from_lr = feats_from_log_reg(X_train[attr_features], Y_train)

In [ ]:
feats_from_lr

##  feature importance for LightGBM

In [ ]:
def get_top_feats_on_importance(booster, name_columns, show_on_plot=50):
    """
    Function plots feature importance and returns features sorted in descending order
    
    booster -  модель
    name_columns - название фичей
    show_on_plot=50 -сколько показывать на графике фичей
    
    """
    
    feature_imp = pd.DataFrame(sorted(zip(booster.feature_importance(importance_type='gain'), name_columns)), columns=['Value','Feature'])

    plt.figure(figsize=(20, 30))
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[:show_on_plot])
    plt.title('LightGBM Features')
    plt.tight_layout()
    plt.show()
    
    return feature_imp.sort_values(by="Value", ascending=False)['Feature'].values


In [ ]:
feats_from_importance = get_top_feats_on_importance(booster_attr, X_train[attr_features].columns, show_on_plot=50)

In [ ]:
feats_from_importance

## shap values

In [ ]:
shap.initjs()
model = booster_attr #model
X = pd.DataFrame(X_train[attr_features]) #выборка
count_show_features = 20 # количество анализируемых фичей

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values[0], X, max_display=count_show_features)

In [ ]:
# Check how toread this chart
shap.summary_plot(shap_values, X, plot_type="bar", max_display=count_show_features)

In [ ]:
# Reorder the features
feature_order = np.argsort(np.sum(np.abs(shap_values[0]), axis=0))
feats_from_shap = X.columns[feature_order][::-1]

In [ ]:
# Dislpays features
feats_from_shap

In [ ]:
gc.collect()

## Plots with differents count of features on Cross-Validation (CV)

In [ ]:
def choose_num_of_feat(params, X, y, list_of_feats, list_of_num_of_feat=None):
    """
    Function builds a chart of model dependecy on number of features

    params - Model parameters
    X - Training Sample
    y - Test sample
    list_of_feats - list of features ordered bytheir importance
    list_of_num_of_feat=None - List of feature counts for which we want to build a chart
    
    """    
    j = 0 
    # We must use ordered by importance features!
    list_of_num_of_feat = list_of_num_of_feat or [1,5, 10,17, 25,35, 50, 70, 85, 100,120,150]
    x_aucs = []
    l = []
    for num_feat in list_of_num_of_feat:
        try:
            feats = list_of_feats[:num_feat]

            cat_feats = get_cat_feats(X[feats], ids=True)
            
            ds = lgb.Dataset(X[feats], #features
                             y, #target
                             categorical_feature=cat_feats)

            cv_res = lgb.cv(params,
                        ds,#что  
                        metrics='auc',
                        num_boost_round=1000,# количество деревьек, если что сработает early stoping 
                        early_stopping_rounds=50,
                        nfold=len(time_folds) if time_folds else NUM_FOLDS,
                        folds=time_folds if time_folds else None,
                        seed=123,
                        categorical_feature=cat_feats)

            auc = cv_res['auc-val-mean'][-1]
            x_aucs.append(auc)
            l.append(num_feat)
            print(j)
            j+=1
            
        except Exception as err:
            print(err)
                
    print(x_aucs)
    plt.figure(figsize=(15,10))
    plt.title('Roc_auc and count_of_features', fontsize=20)
    plt.xlabel('Number of features')
    plt.ylabel('Roc auc score')
    
    plt.plot(l, x_aucs, marker='.', color='green')

    return l[np.argmax(x_aucs)], max(x_aucs)

### log_reg

In [ ]:
# Features for LogReg
choose_num_of_feat(best_params_attr, X_train[attr_features], Y_train, feats_from_lr)

In [ ]:
# Looking at the correlation matrix
plot_heatmap(X_train, feats_from_lr[:50])

### feature_importancefor LGBM

In [ ]:
# фичи по feature importance
choose_num_of_feat(best_params_attr, X_train[attr_features], Y_train, feats_from_importance)

In [ ]:
plot_heatmap(X_train, feats_from_importance[:50])

### shap

In [ ]:
# LGBM features by shap
choose_num_of_feat(best_params_attr, X_train[attr_features], Y_train, feats_from_shap)

In [ ]:
plot_heatmap(X_train, feats_from_shap[:50])

## look through lists of features

In [ ]:
#manual
l=120

In [ ]:
feats_from_importance[:l]

In [ ]:
feats_from_lr[:l]

In [ ]:
feats_from_shap[:l]

## last select features

In [ ]:
#manual
# Selecting a final list of features
features = feats_from_shap[:l]
features

# Model 

In [ ]:
# the third model based on a final list of features
cat_feat_small = get_cat_feats(X_train[features], ids=True)
iteration=0
best_auc_val_so_far=0
best_auc_val_std_so_far=0
best_auc_train_so_far=0
best_auc_train_std_so_far=0
booster_small,best_params_small =run_hyperopt_lgb(X_train[features] ,  Y_train, X_val[features], Y_val, cat_feat_small, evals=10)

In [ ]:
y_pred_train_small = booster_small.predict(X_train[features])
y_pred_val_small = booster_small.predict(X_val[features])

print('roc auc score on train: ', roc_auc_score(Y_train, y_pred_train_small))
print('roc auc score on val: ', roc_auc_score(Y_val, y_pred_val_small))

In [ ]:
ut_ANDREY_v2.print_lift_roc_pr_plot(Y_val, y_pred_val_small,NAME_MODEL='val')
ut_ANDREY_v2.print_lift_roc_pr_plot(Y_train, y_pred_train_small,NAME_MODEL='train')

# Select model 

Here you should choose between 3 models: booster (X_train), booster_attr (X_train[attr_features]),  booster_small (X_train[features])

## performance comparison between models on X_train dataset

In [ ]:
ut_ANDREY_v2.print_lift_roc_pr_plot(Y_train, y_pred_train,NAME_MODEL='all_features')
ut_ANDREY_v2.print_lift_roc_pr_plot(Y_train, y_pred_train_attr,NAME_MODEL='attr')
ut_ANDREY_v2.print_lift_roc_pr_plot(Y_train, y_pred_train_small,NAME_MODEL='last_model')

## performance comparison between models on X_val dataset

In [ ]:
ut_ANDREY_v2.print_lift_roc_pr_plot(Y_val, y_pred_val,NAME_MODEL='all_features')
ut_ANDREY_v2.print_lift_roc_pr_plot(Y_val, y_pred_val_attr,NAME_MODEL='attr')
ut_ANDREY_v2.print_lift_roc_pr_plot(Y_val, y_pred_val_small,NAME_MODEL='last_model')

## Save last choice

In [ ]:
#leave the chosen model only
X_train, X_val = X_train[features], X_val[features]

booster = booster_small
params = best_params_small
cat_feats = cat_feat_small

In [ ]:
gc.collect()

# Report about the selected model

In [ ]:
UMR.print_report_tt(Y_train, y_pred_train_small, Y_val, y_pred_val_small, pd.DataFrame())
plt.show()

In [ ]:
UMR.get_lift_df( y_pred_train_small, Y_train)

# Save files with the model and with the list of selected features

In [ ]:
unique_name='_cl'

In [ ]:
path_to_model = 'lgbm_booster' + unique_name
path_to_features = 'lgbm_booster_features' + unique_name

pickle.dump(booster_small, open(path_to_model, 'wb')) 
pickle.dump(features, open(path_to_features, 'wb'))  